In [ ]:
from calvin_utils.matrix_utilities import find_maximum_voxel, convert_index_to_coordinate
from calvin_utils.generate_nifti import gen_sphere_roi

from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers
import nibabel as nib
import os
import pandas as pd
import glob
import platform
import warnings
warnings.filterwarnings('ignore')
import nilearn

#Imports for selecting location
import tkinter as tk
from tkinter import filedialog


In [ ]:
# Get files to use
use_gui = False
if use_gui == False:
    matrix_path = r'/Users/cu135/Dropbox (Partners HealthCare)/yeo_analysis/to_default_frontop_ventral_seeds/yeo_seeds_only_default_fronto_ventral/Component One Map.nii'
    out_dir = os.path.join(os.path.dirname(matrix_path), f'matrix_maxima/max_voxel.nii')

else: 
    root = tk.Tk();
    root.withdraw();
    matrix_path = filedialog.askopenfilename(title='Please select file to operate on');
    out_dir = filedialog.askdirectory(title='Please select directory to save to');
    out_name = os.path.join(out_dir, (os.path.basename(matrix_path).split('.')[0] + '/max_voxel.nii'))
print('Found matrix at: ', matrix_path)
print('Found output directory at: ', out_dir)


In [ ]:
#Get indices of max voxel
mni_mask_img = nimds.get_img("mni_icbm152")
mni_mask_matrix = mni_mask_img.get_fdata()

matrix_img = image.load_img(matrix_path)
matrix_matrix = matrix_img.get_fdata()
matrix_matrix = np.nan_to_num(matrix_matrix, nan=0, posinf=0, neginf=0)


max_indices = find_maximum_voxel(matrix=matrix_matrix, mask=mni_mask_matrix)
max_indices = [t[0] for t in max_indices]
print(max_indices)

In [ ]:
#Get MNI Coordinates of Max Voxel
from nibabel.affines import apply_affine

coordinate_affine_matrix = mni_mask_img.affine
mni_coords = apply_affine(coordinate_affine_matrix, max_indices)

#Generate the ROI
from calvin_utils.generate_nifti import generate_spherical_roi
sphere_at_maxima = generate_spherical_roi(mni_coords[0], mni_coords[1], mni_coords[2], out_dir=out_dir, radius=2)